In [14]:
from PIL import Image, ImageOps
import pytesseract

# Open the image and apply EXIF orientation if necessary
image = Image.open("/Users/aaryas127/Documents/GitHub/pharmapp/pharmapp-backend/uploads/IMG_8512.jpeg")
image = ImageOps.exif_transpose(image)

# Extract all text from the image
full_text = pytesseract.image_to_string(image)

# Find the position of "Dr." in the text
start_index = full_text.find("Dr.")

# Check if "Dr." was found, and if so, extract text from that position onward
if start_index != -1:
    text = full_text[start_index:]
    print("Extracted Text Starting from 'Dr.':\n", text)
else:
    print("The keyword 'Dr.' was not found in the image.")



Extracted Text Starting from 'Dr.':
 Dr. Germin Attia MD CCFP
Terry Fox Medical

5380 Terry Fox Way #10
Mississauga Ontario L5V OAS
Pract. No.:139270

Tel: 905-858-3030
Fax: 905-858-3031

Written Date: November 5, 2024
AARYA G SHAH DOB:Jan 27, 2003
44 BRETON AVE

MISSISSAUGA, ON L4Z4K2
905-281-2404

Health Ins.#4741252474

VENTOLIN HFA 100@G
1-2 puffs Q6 Hrs PRN
Qty:1 Repeats:0

Ce ee enn celia

—

Signature:

Requesting: Dr. Germin Attia MD CCFP (139270)
MRP: (81264)

Created by: OSCAR The open-source EMR www.oscarcanada.org



In [15]:
# Reading the entire file content
with open("/Users/aaryas127/Documents/GitHub/pharmapp/pharmapp-backend/api_key.txt", 'r') as file:
    content = file.read()
    print(content)

# Reading the file line by line
with open("/Users/aaryas127/Documents/GitHub/pharmapp/pharmapp-backend/api_key.txt", 'r') as file:
    for line in file:
        print(line.strip())


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
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


In [16]:
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI(
  base_url="https://integrate.api.nvidia.com/v1",  # You can change this to OpenAI's URL if needed
  api_key=str(content)
)

# Define the prompt, with the variable 'text' inserted
prompt = f"""
Please extract the following details from the medical text and format them into distinct sections:

**Doctor's Information**
- Name
- Clinic
- Address
- Practitioner Number
- Contact Info

**Patient Details**
- Name
- DOB
- Address
- Contact Info
- Health Insurance Number

**Prescription**
- Medication
- Dosage Instructions
- Quantity
- Repeats

Medical Text:
\"\"\"{text}\"\"\"
"""

# Request completion from OpenAI's model
completion = client.chat.completions.create(
  model="meta/llama-3.2-3b-instruct",  # You can switch this model to one compatible with OpenAI if needed
  messages=[{"role": "user", "content": prompt}],
  temperature=0.2,
  top_p=0.7,
  max_tokens=1024,
  stream=True
)

# Collect and print the response as it's streamed
for chunk in completion:
    if chunk.choices[0].delta.content is not None:
        output = chunk.choices[0].delta.content
        print(output, end="")


Here are the extracted details formatted into distinct sections:

**Doctor's Information**

* Name: Dr. Germin Attia MD CCFP
* Clinic: Terry Fox Medical
* Address: 5380 Terry Fox Way #10, Mississauga, Ontario L5V OAS
* Practitioner Number: 139270
* Contact Info: Tel: 905-858-3030, Fax: 905-858-3031

**Patient Details**

* Name: Aaryah G Shah
* Date of Birth (DOB): January 27, 2003
* Address: 44 Breton Ave, Mississauga, ON L4Z4K2
* Contact Info: Tel: 905-281-2404
* Health Insurance Number: 4741252474

**Prescription**

* Medication: Ventolin HFA 100mcg
* Dosage Instructions: 1-2 puffs Q6 hours PRN
* Quantity: 1
* Repeats: 0